In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import gensim
import os
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [2]:
df = pd.read_csv('classification_data.csv')

In [3]:
df.head()

,Unnamed: 0,Category_id,Category,Content
0,0,0,Unrest,GANDHINAGAR: The state government's vaunted 'K...
1,1,0,Unrest,"LUCKNOW: A committed RSS pracharak since 1977,..."
2,2,0,Unrest,"CHENNAI: As news of the violence spread, the a..."
3,3,0,Unrest,BHIWANDI: Three boys have been rescued from th...
4,4,0,Unrest,CHANDIGARH: As chief of internal security and ...


In [4]:
data = df['Content']

In [5]:
data = data.tolist()

In [6]:
stop_words = set(stopwords.words('english'))

In [7]:
filtered_data = []
for i,d in enumerate(data):
    filtered_sentence = [w for w in word_tokenize(str(d)) if not w in stop_words]
    filtered_data.append(''.join(filtered_sentence))

In [8]:
data = filtered_data

In [9]:
tagged_data = [TaggedDocument(words=word_tokenize(str(_d).lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [10]:
max_epochs = 40
vec_size = 1000
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_1000.model")
print("Model Saved")

/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
Model Saved


In [14]:
model = Doc2Vec.load("d2v.model")

In [ ]:
test_data = word_tokenize("I love chatbots".lower())

In [ ]:
v1 = model.infer_vector(test_data)

In [21]:
import allennlp
from allennlp.modules.elmo import Elmo, batch_to_ids

In [12]:
options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

elmo = Elmo(options_file, weight_file, 2, dropout=0)

100%|██████████| 374434792/374434792 [01:07<00:00, 5512642.59B/s]


In [15]:
test = df.loc[0]['Content']

In [16]:
filtered_test = []

In [17]:
filtered_sentence = [w for w in word_tokenize(str(test)) if not w in stop_words]
filtered_test.append(''.join(filtered_sentence))

In [23]:
character_ids = batch_to_ids(filtered_sentence)

In [24]:
embeddings = elmo(character_ids)

In [33]:
embeddings['elmo_representations'][0]

1024